In [1]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [143]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [165]:
df = pd.read_csv('./train.csv/train.csv')

df.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price
0,1030324,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013.0,NaN,09-dec-2013,luxury sedan,"parf car, premium ad car, low mileage car",...,73000.0,45330.0,50462.0,NaN,NaN,uncategorized,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",NaN,71300.0
1,1021510,Toyota Hiace 3.0M,NaN,hiace,high loan available! low mileage unit. wear an...,2014.0,NaN,26-jan-2015,van,premium ad car,...,110112.0,27502.0,1376.0,NaN,25-jan-2035,uncategorized,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,NaN,43800.0
2,1026909,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016.0,NaN,25-jul-2016,luxury sedan,"parf car, premium ad car",...,80000.0,27886.0,26041.0,NaN,NaN,uncategorized,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,NaN,95500.0
3,1019371,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019.0,NaN,17-nov-2020,luxury sedan,"parf car, almost new car, consignment car",...,9800.0,46412.0,56977.0,NaN,NaN,uncategorized,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,NaN,197900.0
4,1031014,Honda Civic 1.6A VTi,NaN,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019.0,NaN,20-sep-2019,mid-sized sedan,parf car,...,40000.0,20072.0,20101.0,NaN,NaN,uncategorized,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",NaN,103200.0


In [166]:
print(df.shape)
print(df.columns.to_list())
print(df.iloc[0])

(16784, 30)
['listing_id', 'title', 'make', 'model', 'description', 'manufactured', 'original_reg_date', 'reg_date', 'type_of_vehicle', 'category', 'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap', 'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value', 'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category', 'features', 'accessories', 'indicative_price', 'price']
listing_id                                                     1030324
title                           BMW 3 Series 320i Gran Turismo M-Sport
make                                                               bmw
model                                                             320i
description          1 owner! 320i gt m-sports model! big brake kit...
manufactured                                                    2013.0
original_reg_date                                                  NaN
reg_date                                                   09-dec-2013
type_of_vehicle             

In [167]:
# car brands. to-do: aggregate and convert to ordinal label
make_names = np.unique(temp)
print(make_names)

['alfa romeo' 'alpine' 'aston martin' 'audi' 'austin' 'bentley' 'bmw'
 'byd' 'chery' 'chevrolet' 'chrysler' 'citroen' 'cupra' 'daf' 'daihatsu'
 'daimler' 'dodge' 'dongfeng' 'ferrari' 'fiat' 'ford' 'foton'
 'golden dragon' 'hafei' 'higer' 'hino' 'honda' 'hummer' 'hyundai'
 'infiniti' 'international' 'isuzu' 'iveco' 'jaguar' 'jeep' 'joylong'
 'kia' 'lamborghini' 'land rover' 'lexus' 'lotus' 'man' 'maserati' 'maxus'
 'mazda' 'mclaren' 'mercedes' 'mercedes-benz' 'mg' 'mini' 'mitsubishi'
 'mitsuoka' 'morris' 'nissan' 'opel' 'perodua' 'peugeot' 'porsche'
 'proton' 'renault' 'riley' 'rolls-royce' 'rover' 'ruf' 'saab' 'scania'
 'seat' 'skoda' 'ssangyong' 'subaru' 'suzuki' 'tesla' 'toyota' 'ud'
 'volkswagen' 'volvo' 'yutong']


In [168]:
missing_make = np.array(df['make'].isnull())
print(missing_make.sum())

2160


In [169]:
missing_owner = np.array(df['no_of_owners'].isnull())
def extract_owners(d):
    ds = d.split(' ')
    i = -1
    if 'owner' in ds:
        i = ds.index('owner')
    elif 'owners' in ds:
        i = ds.index('owners')
    
    amt = False
    if i != -1:
        amt = ds[i - 1]
    if amt == 'one' or amt == '1' or amt == 'original':
        amt = 1
    elif amt == 'two' or amt == '2':
        amt = 2
    else:
        amt = False
    if amt:
        amt = int(amt)
    return amt

df['description'] = df['description'].str.lower().replace()
owner = np.array(df['description'].map(extract_owners), dtype=bool)
remains = np.logical_and(owner, missing_owner)
df['no_of_owners'][remains] = owner[remains]
print(remains.sum()) # orz...

3


G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [170]:
def extract_model(de):
    model_list = ['alfa romeo', 'alpine', 'aston martin', 'audi', 'austin', 'bentley', 'bmw',
 'byd', 'chery', 'chevrolet', 'chrysler', 'citroen', 'cupra', 'daf', 'daihatsu',
 'daimler', 'dodge', 'dongfeng', 'ferrari', 'fiat', 'ford', 'foton',
 'golden dragon', 'hafei', 'higer', 'hino', 'honda', 'hummer', 'hyundai',
 'infiniti', 'international', 'isuzu', 'iveco', 'jaguar', 'jeep', 'joylong',
 'kia', 'lamborghini', 'land rover', 'lexus', 'lotus', 'man', 'maserati', 'maxus',
 'mazda', 'mclaren', 'mercedes', 'mercedes-benz', 'mg', 'mini', 'mitsubishi',
 'mitsuoka', 'morris', 'nissan', 'opel', 'perodua', 'peugeot', 'porsche',
 'proton', 'renault', 'riley', 'rolls-royce', 'rover', 'ruf', 'saab', 'scania',
 'seat', 'skoda', 'ssangyong', 'subaru', 'suzuki', 'tesla', 'toyota', 'ud',
 'volkswagen', 'volvo', 'yutong']
    des = de.split(' ')
    for d in des:
        if d in model_list:
            return d
    return False

df['title'] = df['title'].str.lower().replace('-', '')
make_from_titles = df['title'].map(extract_model)
b = np.array(make_from_titles, dtype=bool)
remains = np.logical_and(b, missing_make)
df['make'][remains] = make_from_titles[remains]
#2103 make filled

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [171]:
df = df.drop(columns = ['listing_id', 'eco_category', 'accessories', 'curb_weight'])

In [172]:
model_names = np.unique(df['model'])
print(model_names)
#things appear ok. need to aggregate

['107' '116d' '116i' '118i' '120i' '125i' '12c' '1300' '135i' '14.280'
 '159' '190e' '1m' '2' '200' '2008' '200sx' '207cc' '208' '216' '216d'
 '216i' '218d' '218i' '220' '220i' '225xe' '228i' '230' '230i' '250'
 '280s' '3' '3008' '300c' '300ce' '300gd' '300sel' '300sl' '308' '315'
 '316i' '318ci' '318i' '320ce' '320d' '320i' '323i' '325ci' '325i' '328i'
 '330ci' '330e' '330i' '335i' '340i' '348' '360' '3800s' '420i' '428i'
 '430i' '435i' '440i' '450slc' '458' '488' '500' '5008' '508' '520d'
 '520i' '523i' '525i' '528i' '530e' '530i' '535i' '540i' '570s' '575m'
 '599' '600lt' '630ci' '630i' '640i' '650i' '650s' '718' '720s' '730d'
 '730i' '730li' '740e' '740i' '740li' '750li' '840i' '850i' '86' '9-3'
 '9-5' '900s' '911' '924' '944' 'a1' 'a110' 'a156' 'a180' 'a200' 'a250'
 'a3' 'a35' 'a4' 'a45' 'a5' 'a6' 'a7' 'a8' 'accent' 'accord'
 'activehybrid' 'actros' 'actyon' 'airwave' 'alhambra' 'allion' 'almera'
 'alphard' 'alpina' 'amg' 'apv' 'aqua' 'arona' 'arteon' 'astra' 'asx'
 'ateca' 'atlas

In [173]:
print(df['manufactured'].sample(25))

11436    2007.0
6034     2009.0
8813     2017.0
15836    2019.0
12462    2008.0
5500     2012.0
304      2017.0
16752    2011.0
3902     2017.0
11316    2014.0
3264     2012.0
988      2019.0
13623    2016.0
15092    2008.0
7101     2007.0
7665     2015.0
2129     2018.0
9493     2019.0
9012     2015.0
4968     2007.0
9072     2017.0
12928    2020.0
8639     2017.0
11068    2020.0
7555     2010.0
Name: manufactured, dtype: float64


In [174]:
d1 = np.array(df['manufactured'].isnull())
df['age'] = df['manufactured'].copy()
df['age'][d1] = df['original_reg_date'][d1]
d1 = df['age'].isnull()
df['age'][d1] = df['reg_date'][d1]

def extract_year(date):
    date = str(date)
    if len(date) != 4:
        if '.' in date and len(date) == 6:
            year = date[:-2]
        else:
            l = date.split('-')
            year = l[-1]
    return int(year)
df['age'] = 2021 - df['age'].map(extract_year)

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [175]:
# take out outliers
df['age'] = df['age'][df.age < 30]
df['age'] = df['age'][df.age >= 0]

In [184]:
df['fuel_type'] = df['fuel_type'].fillna('petrol')

In [185]:
df.head()

,title,make,model,manufactured,type_of_vehicle,category,transmission,power,fuel_type,engine_cap,...,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,indicative_price,price,age
0,bmw 3 series 320i gran turismo m-sport,bmw,320i,2013.0,luxury sedan,"parf car, premium ad car, low mileage car",auto,135.0,petrol,1997.0,...,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,NaN,NaN,71300.0,8.0
1,toyota hiace 3.0m,toyota,hiace,2014.0,van,premium ad car,manual,NaN,diesel,2982.0,...,NaN,3648.0,110112.0,27502.0,1376.0,NaN,25-jan-2035,NaN,43800.0,7.0
2,mercedes-benz cla-class cla180,mercedes-benz,cla180,2016.0,luxury sedan,"parf car, premium ad car",auto,90.0,petrol,1595.0,...,740.0,44517.0,80000.0,27886.0,26041.0,NaN,NaN,NaN,95500.0,5.0
3,mercedes-benz e-class e180 avantgarde,mercedes-benz,e180,2019.0,luxury sedan,"parf car, almost new car, consignment car",auto,115.0,petrol,1497.0,...,684.0,80301.0,9800.0,46412.0,56977.0,NaN,NaN,NaN,197900.0,2.0
4,honda civic 1.6a vti,honda,civic,2019.0,mid-sized sedan,parf car,auto,92.0,petrol,1597.0,...,742.0,36453.0,40000.0,20072.0,20101.0,NaN,NaN,NaN,103200.0,2.0


In [182]:
df = df.drop(columns = ['description', 'original_reg_date', 'reg_date', 'features', 'lifespan'])

In [183]:
df.head()

,title,make,model,manufactured,type_of_vehicle,category,transmission,power,fuel_type,engine_cap,...,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,indicative_price,price,age
0,bmw 3 series 320i gran turismo m-sport,bmw,320i,2013.0,luxury sedan,"parf car, premium ad car, low mileage car",auto,135.0,NaN,1997.0,...,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,NaN,NaN,71300.0,8.0
1,toyota hiace 3.0m,toyota,hiace,2014.0,van,premium ad car,manual,NaN,diesel,2982.0,...,NaN,3648.0,110112.0,27502.0,1376.0,NaN,25-jan-2035,NaN,43800.0,7.0
2,mercedes-benz cla-class cla180,mercedes-benz,cla180,2016.0,luxury sedan,"parf car, premium ad car",auto,90.0,NaN,1595.0,...,740.0,44517.0,80000.0,27886.0,26041.0,NaN,NaN,NaN,95500.0,5.0
3,mercedes-benz e-class e180 avantgarde,mercedes-benz,e180,2019.0,luxury sedan,"parf car, almost new car, consignment car",auto,115.0,NaN,1497.0,...,684.0,80301.0,9800.0,46412.0,56977.0,NaN,NaN,NaN,197900.0,2.0
4,honda civic 1.6a vti,honda,civic,2019.0,mid-sized sedan,parf car,auto,92.0,NaN,1597.0,...,742.0,36453.0,40000.0,20072.0,20101.0,NaN,NaN,NaN,103200.0,2.0


In [180]:
# remove useless atttributes
# deal with missing values
    # convert feature / description and title to fill in missing values for model and make
    # drop original reg date / reg date after using it to fill missing manufactured date, convert to age
    # drop these values afterwards
    # if fuel type is NA, change to petral, which is the default type
# explore remaining attributes
    # standardize, normalize
    # simplify, aggregate
    # one-hot encoding, convert to numerical label
    # remove outliers
# find out other things to clean

# qn: indicative price more representative? combine with retail price?